In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10

In [3]:
#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping,ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [4]:
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
np.random.seed(1337) # The seed I used - pick your own or comment out for a random seed. A constant seed allows for better comparisons though

In [5]:
input_path = "F:/Kaggle/Statoil/inputs/"
batch_size = 32

In [6]:
train = pd.read_json(input_path+"train.json")
target_train=train['is_iceberg']
test = pd.read_json(input_path+"test.json")

In [7]:
def iso(arr):
    p = np.reshape(np.array(arr), [75,75]) >(np.mean(np.array(arr))+2*np.std(np.array(arr)))
    return p * np.reshape(np.array(arr), [75,75])
def size(arr):     
    return float(np.sum(arr<-5))/(75*75)

In [8]:
### feat engg
train['iso1'] = train.iloc[:, 0].apply(iso)
train['iso2'] = train.iloc[:, 1].apply(iso)
test['iso1'] = test.iloc[:, 0].apply(iso)
test['iso2'] = test.iloc[:, 1].apply(iso)

# Feature engineering s1 s2 and size.
train['s1'] = train.iloc[:,5].apply(size)
train['s2'] = train.iloc[:,6].apply(size)
test['s1'] = test['iso1'].apply(size)
test['s2'] = test['iso2'].apply(size)

train['band_1'] = train['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
train['band_2'] = train['band_2'].apply(lambda x: np.array(x).reshape(75, 75))
test['band_1'] = test['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
test['band_2'] = test['band_2'].apply(lambda x: np.array(x).reshape(75, 75))

train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')


#####process test set!
band_1_test = np.concatenate([im for im in test['band_1']]).reshape(-1, 75, 75)
band_2_test = np.concatenate([im for im in test['band_2']]).reshape(-1, 75, 75)
full_img_test = np.stack([band_1_test, band_2_test,(band_1_test+band_2_test)/2], axis=1)
angle_test=test['inc_angle']
size_test=test['s1']
test['is_iceberg']=0

In [9]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [10]:
### concat angle and size
Tr_s1 = train['s1']
Tr_s2 = train['s2']
Te_s1 = test['s1']
Te_s2 = test['s2']

In [11]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [12]:
X_train = get_scaled_imgs(train)
X_test = get_scaled_imgs(test)

In [13]:
print("shape of train: {} and test: {}" .format(X_train.shape,X_test.shape))

shape of train: (1604, 75, 75, 3) and test: (8424, 75, 75, 3)


In [14]:
from clr_callback import *

In [15]:
#### updated to include size and other variables

# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays

def gen_flow_for_two_inputs(X1, X2, s1,s2,y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    genX3 = gen.flow(X1,s1, batch_size=batch_size,seed=55)
    genX4 = gen.flow(X1,s2, batch_size=batch_size,seed=55)

    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            X4i = genX4.next()
            
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], (np.vstack((X2i[1],X3i[1],X4i[1])).T)], X1i[1]

# Finally create generator
# Finally create generator
def get_callbacks(filepath, patience=2):
    #es = EarlyStopping('val_loss', patience=10, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.01)
    #clr_triangular = CyclicLR(mode='triangular')
    clr_Cyclic = CyclicLR(mode='exp_range', gamma=0.99994)
    return [msave,clr_Cyclic]

In [16]:
# # Define the image transformations here
# gen = ImageDataGenerator(horizontal_flip = True,
#                          vertical_flip = True,
#                          width_shift_range = 0.,
#                          height_shift_range = 0.,
#                          channel_shift_range=0,
#                          zoom_range = 0.2,
#                          rotation_range = 10)

# # Here is the function that merges our two generators
# # We use the exact same generator with the same random seed for both the y and angle arrays
# def gen_flow_for_two_inputs(X1, X2, y):
#     genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
#     genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
#     while True:
#             X1i = genX1.next()
#             X2i = genX2.next()
#             #Assert arrays are equal - this was for peace of mind, but slows down training
#             #np.testing.assert_array_equal(X1i[0],X2i[0])
#             yield [X1i[0], X2i[1]], X1i[1]

# def gen_flow_for_test(X1, X2,y=None):
#     genX12 = gen.flow(X1,y, batch_size=batch_size,seed=55)
#     genX22 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
#     while True:
#             X21i = genX12.next()
#             X22i = genX22.next()
#             #Assert arrays are equal - this was for peace of mind, but slows down training
#             #np.testing.assert_array_equal(X1i[0],X2i[0])
#             yield [X21i[0], X22i[1]]
            
# # Finally create generator
# def get_callbacks(filepath, patience=2):
#     #es = EarlyStopping('val_loss', patience=10, mode="min")
#     msave = ModelCheckpoint(filepath, save_best_only=True)
#     #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.01)
#     #clr_triangular = CyclicLR(mode='triangular')
#     clr_Cyclic = CyclicLR(mode='exp_range', gamma=0.99994)
#     return [msave,clr_Cyclic]

In [17]:
def getVggAngleModel():
    input_2 = Input(shape=[3,], name="angle")
    angle_layer = Dense(1,activation='relu')(input_2)
    
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [18]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle,Tr_s1,Tr_s2, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle +size
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]
        
        #size
        X_size1_cv = Tr_s1[train_idx]
        X_size2_cv = Tr_s2[train_idx]
        X_size1_hold = Tr_s1[test_idx]
        X_size2_hold = Tr_s2[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug2_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv,X_size1_cv,X_size2_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout),
                callbacks=callbacks)
        

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,np.vstack((X_angle_cv,X_size1_cv,X_size2_cv)).T], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, np.vstack((X_test_angle,Te_s1,Te_s2)).T])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, np.vstack((X_angle,Tr_s1,Tr_s2)).T])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [19]:
batch_size = 32
preds=myAngleCV(X_train, X_angle, Tr_s1,Tr_s2,X_test)


===================FOLD= 0


C:\Users\ADMIN\Anaconda3\envs\Pytorch\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - 7s 291ms/step - loss: 0.6703 - acc: 0.5794 - val_loss: 0.5509 - val_acc: 0.7701
Epoch 2/100
24/24 [==============================] - 4s 180ms/step - loss: 0.5692 - acc: 0.7229 - val_loss: 0.4464 - val_acc: 0.7850
Epoch 3/100
24/24 [==============================] - 5s 223ms/step - loss: 0.4813 - acc: 0.7646 - val_loss: 0.3526 - val_acc: 0.8430
Epoch 4/100
24/24 [==============================] - 5s 200ms/step - loss: 0.4074 - acc: 0.8151 - val_loss: 0.2973 - val_acc: 0.8411
Epoch 5/100
24/24 [==============================] - 5s 196ms/step - loss: 0.3843 - acc: 0.8251 - val_loss: 0.3722 - val_acc: 0.8486
Epoch 6/100
24/24 [==============================] - 5s 227ms/step - loss: 0.3915 - acc: 0.8128 - val_loss: 0.2684 - val_acc: 0.8860
Epoch 7/100
24/24 [==============================] - 4s 185ms/step - loss: 0.3446 - acc: 0.8477 - val_loss: 0.2415 - val_acc: 0.8916
Epoch 8/100
24/24 [==============================] - 4s 171ms/step - 

24/24 [==============================] - 4s 186ms/step - loss: 0.1778 - acc: 0.9219 - val_loss: 0.2306 - val_acc: 0.9121
Epoch 63/100
24/24 [==============================] - 4s 170ms/step - loss: 0.1936 - acc: 0.9304 - val_loss: 0.2153 - val_acc: 0.9028
Epoch 64/100
24/24 [==============================] - 4s 168ms/step - loss: 0.2193 - acc: 0.9101 - val_loss: 0.4051 - val_acc: 0.8486
Epoch 65/100
24/24 [==============================] - 4s 176ms/step - loss: 0.2595 - acc: 0.9076 - val_loss: 0.2219 - val_acc: 0.9065
Epoch 66/100
24/24 [==============================] - 4s 169ms/step - loss: 0.1927 - acc: 0.9309 - val_loss: 0.2405 - val_acc: 0.9047
Epoch 67/100
24/24 [==============================] - 4s 169ms/step - loss: 0.2265 - acc: 0.9137 - val_loss: 0.2562 - val_acc: 0.8972
Epoch 68/100
24/24 [==============================] - 4s 175ms/step - loss: 0.1805 - acc: 0.9304 - val_loss: 0.2449 - val_acc: 0.9084
Epoch 69/100
24/24 [==============================] - 4s 183ms/step - loss:

24/24 [==============================] - 4s 172ms/step - loss: 0.2333 - acc: 0.8939 - val_loss: 0.2460 - val_acc: 0.9047
Epoch 23/100
24/24 [==============================] - 4s 167ms/step - loss: 0.2620 - acc: 0.8978 - val_loss: 0.2317 - val_acc: 0.8953
Epoch 24/100
24/24 [==============================] - 4s 187ms/step - loss: 0.2276 - acc: 0.9049 - val_loss: 0.2500 - val_acc: 0.9084
Epoch 25/100
24/24 [==============================] - 4s 180ms/step - loss: 0.2547 - acc: 0.8952 - val_loss: 0.1911 - val_acc: 0.9196
Epoch 26/100
24/24 [==============================] - 5s 199ms/step - loss: 0.2660 - acc: 0.8869 - val_loss: 0.2346 - val_acc: 0.9065
Epoch 27/100
24/24 [==============================] - 5s 191ms/step - loss: 0.2154 - acc: 0.8978 - val_loss: 0.1953 - val_acc: 0.9215
Epoch 28/100
24/24 [==============================] - 5s 188ms/step - loss: 0.2420 - acc: 0.9023 - val_loss: 0.1914 - val_acc: 0.9159
Epoch 29/100
24/24 [==============================] - 4s 184ms/step - loss:

24/24 [==============================] - 4s 184ms/step - loss: 0.1454 - acc: 0.9466 - val_loss: 0.2470 - val_acc: 0.9084
Epoch 84/100
24/24 [==============================] - 4s 177ms/step - loss: 0.1724 - acc: 0.9382 - val_loss: 0.2015 - val_acc: 0.9140
Epoch 85/100
24/24 [==============================] - 4s 186ms/step - loss: 0.1394 - acc: 0.9531 - val_loss: 0.3002 - val_acc: 0.8935
Epoch 86/100
24/24 [==============================] - 4s 177ms/step - loss: 0.1709 - acc: 0.9362 - val_loss: 0.2349 - val_acc: 0.8897
Epoch 87/100
24/24 [==============================] - 4s 167ms/step - loss: 0.1253 - acc: 0.9526 - val_loss: 0.3148 - val_acc: 0.8879
Epoch 88/100
24/24 [==============================] - 4s 166ms/step - loss: 0.1347 - acc: 0.9505 - val_loss: 0.2714 - val_acc: 0.8972
Epoch 89/100
24/24 [==============================] - 4s 176ms/step - loss: 0.1547 - acc: 0.9453 - val_loss: 0.2784 - val_acc: 0.8879
Epoch 90/100
24/24 [==============================] - 4s 169ms/step - loss:

24/24 [==============================] - 4s 172ms/step - loss: 0.1905 - acc: 0.9251 - val_loss: 0.2988 - val_acc: 0.8558
Epoch 44/100
24/24 [==============================] - 4s 168ms/step - loss: 0.2115 - acc: 0.9059 - val_loss: 0.2558 - val_acc: 0.9045
Epoch 45/100
24/24 [==============================] - 4s 179ms/step - loss: 0.1812 - acc: 0.9232 - val_loss: 0.2786 - val_acc: 0.8858
Epoch 46/100
24/24 [==============================] - 4s 169ms/step - loss: 0.1638 - acc: 0.9309 - val_loss: 0.2883 - val_acc: 0.8801
Epoch 47/100
24/24 [==============================] - 4s 171ms/step - loss: 0.1790 - acc: 0.9316 - val_loss: 0.3138 - val_acc: 0.8502
Epoch 48/100
24/24 [==============================] - 4s 173ms/step - loss: 0.1852 - acc: 0.9076 - val_loss: 0.2746 - val_acc: 0.8839
Epoch 49/100
24/24 [==============================] - 4s 187ms/step - loss: 0.1513 - acc: 0.9400 - val_loss: 0.3007 - val_acc: 0.8858
Epoch 50/100
24/24 [==============================] - 4s 185ms/step - loss:

### Pseudo labels

In [37]:
### choose i to maximize best choices for pseudo labels based on prob output
i = 0.049
idx_pred_1 = (np.where(preds>0.95+i))
idx_pred_0 = (np.where(preds<0.05-i))

In [ ]:
# ### concat angle and size

# Tr_s1 = train['s1']
# Tr_s2 = train['s2']
# Te_s1 = test['s1'] = test['s1']
# Te_s2 = test['s2']

In [23]:
X_.head()

0    34.966400
1    32.615072
2    37.505433
3    34.473900
4    43.918874
Name: inc_angle, dtype: float64

In [38]:
### add pseudolabels data to original train 
add_test = X_test[idx_pred_1]
add_angle = X_test_angle.iloc[idx_pred_1]
add_test_s1 = Te_s1.iloc[idx_pred_1]
add_test_s2 = Te_s2.iloc[idx_pred_1]

add_test1=X_test[idx_pred_0]
add_angle1 = X_test_angle.iloc[idx_pred_0]
add_test1_s1 = Te_s1.iloc[idx_pred_0]
add_test1_s2 = Te_s2.iloc[idx_pred_0]


additinal_test = np.concatenate((add_test,add_test1),axis=0)
additional_angle = np.concatenate((add_angle,add_angle1),axis=0)
additional_s1 = np.concatenate((add_test_s1,add_test1_s1),axis=0)
additional_s2 = np.concatenate((add_test_s2,add_test1_s2),axis=0)

additional_train = np.concatenate((X_train,additinal_test),axis=0)
additional_angle = np.concatenate((X_angle,additional_angle),axis=0)
additional_s1_tr = np.concatenate((Tr_s1,additional_s1),axis=0)
additional_s2_tr = np.concatenate((Tr_s2,additional_s2),axis=0)


Y_temp1 = [1]*add_test.shape[0]
Y_temp0 = [0]*add_test1.shape[0]

Ynew = np.concatenate((Y_temp1,Y_temp0),axis=0)
additionalY = np.concatenate((target_train,Ynew),axis=0)

In [39]:
additionalY.shape[0]-target_train.shape[0]

575

In [47]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV_pseud(additional_train, additional_angle,additional_s1_tr,additional_s2_tr, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(additional_train, additionalY))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*additionalY
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = additional_train[train_idx]
        y_train_cv = additionalY[train_idx]
        X_holdout = additional_train[test_idx]
        Y_holdout= additionalY[test_idx]
        
        #Angle +size
        X_angle_cv = additional_angle[train_idx]
        X_angle_hold = additional_angle[test_idx]
        
        #size
        X_size1_cv = additional_s1_tr[train_idx]
        X_size2_cv = additional_s2_tr[train_idx]
        X_size1_hold = additional_s1_tr[test_idx]
        X_size2_hold = additional_s2_tr[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug2_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv,X_size1_cv,X_size2_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=50,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout),
                callbacks=callbacks)
        

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,np.vstack((X_angle_cv,X_size1_cv,X_size2_cv)).T], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, np.vstack((X_test_angle,Te_s1,Te_s2)).T])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([additional_train, np.vstack((additional_angle,additional_s1_tr,additional_s2_tr)).T])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(additionalY, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(additionalY, y_valid_pred_log))
    return y_test_pred_log

In [48]:
batch_size = 32
predicition_pesudo=myAngleCV_pseud(additional_train, additional_angle,additional_s1_tr,additional_s2_tr, X_test)


===================FOLD= 0


C:\Users\ADMIN\Anaconda3\envs\Pytorch\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/50
24/24 [==============================] - 6s 249ms/step - loss: 0.6934 - acc: 0.6003 - val_loss: 0.9137 - val_acc: 0.5103
Epoch 2/50
24/24 [==============================] - 4s 172ms/step - loss: 0.5109 - acc: 0.7344 - val_loss: 0.3119 - val_acc: 0.8467
Epoch 3/50
24/24 [==============================] - 4s 183ms/step - loss: 0.3612 - acc: 0.8185 - val_loss: 0.2568 - val_acc: 0.8513
Epoch 4/50
24/24 [==============================] - 4s 183ms/step - loss: 0.2967 - acc: 0.8568 - val_loss: 0.2134 - val_acc: 0.8833
Epoch 5/50
24/24 [==============================] - 4s 170ms/step - loss: 0.3056 - acc: 0.8381 - val_loss: 0.3201 - val_acc: 0.8101
Epoch 6/50
24/24 [==============================] - 5s 188ms/step - loss: 0.2607 - acc: 0.8815 - val_loss: 0.2764 - val_acc: 0.8673
Epoch 7/50
24/24 [==============================] - 5s 208ms/step - loss: 0.3003 - acc: 0.8479 - val_loss: 0.1941 - val_acc: 0.9039
Epoch 8/50
24/24 [==============================] - 4s 184ms/step - loss: 0.

24/24 [==============================] - 4s 173ms/step - loss: 0.2386 - acc: 0.8854 - val_loss: 0.1696 - val_acc: 0.9130
Epoch 12/50
24/24 [==============================] - 4s 175ms/step - loss: 0.2457 - acc: 0.8824 - val_loss: 0.1639 - val_acc: 0.9153
Epoch 13/50
24/24 [==============================] - 4s 182ms/step - loss: 0.2226 - acc: 0.8932 - val_loss: 0.1466 - val_acc: 0.9314
Epoch 14/50
24/24 [==============================] - 4s 179ms/step - loss: 0.2238 - acc: 0.8818 - val_loss: 0.1502 - val_acc: 0.9291
Epoch 15/50
24/24 [==============================] - 5s 188ms/step - loss: 0.2154 - acc: 0.9036 - val_loss: 0.1445 - val_acc: 0.9336
Epoch 16/50
24/24 [==============================] - 5s 190ms/step - loss: 0.2262 - acc: 0.8776 - val_loss: 0.1467 - val_acc: 0.9245
Epoch 17/50
24/24 [==============================] - 5s 190ms/step - loss: 0.1865 - acc: 0.9092 - val_loss: 0.1906 - val_acc: 0.9062
Epoch 18/50
24/24 [==============================] - 4s 170ms/step - loss: 0.2468

Epoch 22/50
24/24 [==============================] - 4s 167ms/step - loss: 0.1866 - acc: 0.9297 - val_loss: 0.1654 - val_acc: 0.9333
Epoch 23/50
24/24 [==============================] - 4s 169ms/step - loss: 0.2171 - acc: 0.9050 - val_loss: 0.1810 - val_acc: 0.9034
Epoch 24/50
24/24 [==============================] - 5s 188ms/step - loss: 0.2156 - acc: 0.9128 - val_loss: 0.1555 - val_acc: 0.9310
Epoch 25/50
24/24 [==============================] - 4s 181ms/step - loss: 0.1880 - acc: 0.9141 - val_loss: 0.2015 - val_acc: 0.9126
Epoch 26/50
24/24 [==============================] - 5s 188ms/step - loss: 0.1681 - acc: 0.9245 - val_loss: 0.1541 - val_acc: 0.9356
Epoch 27/50
24/24 [==============================] - 4s 186ms/step - loss: 0.1886 - acc: 0.9089 - val_loss: 0.1708 - val_acc: 0.9172
Epoch 28/50
24/24 [==============================] - 4s 169ms/step - loss: 0.1866 - acc: 0.9153 - val_loss: 0.1755 - val_acc: 0.9172
Epoch 29/50
24/24 [==============================] - 4s 171ms/step - 

24/24 [==============================] - 4s 182ms/step - loss: 0.1804 - acc: 0.9219 - val_loss: 0.2064 - val_acc: 0.9126
Epoch 33/50
24/24 [==============================] - 4s 176ms/step - loss: 0.1803 - acc: 0.9284 - val_loss: 0.2149 - val_acc: 0.9149
Epoch 34/50
24/24 [==============================] - 4s 187ms/step - loss: 0.1611 - acc: 0.9336 - val_loss: 0.2231 - val_acc: 0.9126
Epoch 35/50
24/24 [==============================] - 4s 187ms/step - loss: 0.1535 - acc: 0.9218 - val_loss: 0.2628 - val_acc: 0.9080
Epoch 36/50
24/24 [==============================] - 4s 173ms/step - loss: 0.1815 - acc: 0.9284 - val_loss: 0.2100 - val_acc: 0.9218
Epoch 37/50
24/24 [==============================] - 5s 204ms/step - loss: 0.2064 - acc: 0.9153 - val_loss: 0.2080 - val_acc: 0.9080
Epoch 38/50
24/24 [==============================] - 5s 203ms/step - loss: 0.1572 - acc: 0.9284 - val_loss: 0.1987 - val_acc: 0.9080
Epoch 39/50
24/24 [==============================] - 4s 180ms/step - loss: 0.1704

Epoch 43/50
24/24 [==============================] - 4s 169ms/step - loss: 0.1750 - acc: 0.9245 - val_loss: 0.2069 - val_acc: 0.9172
Epoch 44/50
24/24 [==============================] - 5s 188ms/step - loss: 0.1605 - acc: 0.9336 - val_loss: 0.2060 - val_acc: 0.9218
Epoch 45/50
24/24 [==============================] - 5s 213ms/step - loss: 0.1691 - acc: 0.9375 - val_loss: 0.1688 - val_acc: 0.9218
Epoch 46/50
24/24 [==============================] - 5s 206ms/step - loss: 0.1671 - acc: 0.9193 - val_loss: 0.2034 - val_acc: 0.9149
Epoch 47/50
24/24 [==============================] - 5s 200ms/step - loss: 0.1766 - acc: 0.9336 - val_loss: 0.1956 - val_acc: 0.9218
Epoch 48/50
24/24 [==============================] - 4s 173ms/step - loss: 0.1537 - acc: 0.9388 - val_loss: 0.2211 - val_acc: 0.9080
Epoch 49/50
24/24 [==============================] - 4s 170ms/step - loss: 0.1576 - acc: 0.9323 - val_loss: 0.1800 - val_acc: 0.9310
Epoch 50/50
24/24 [==============================] - 4s 176ms/step - 

In [49]:
#Submission
import datetime as dt

submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicition_pesudo
submission.to_csv('F:/Kaggle/Statoil/submit/sub5cv_originalvgg16_pseudolabels_featengg'+dt.datetime.today().strftime("%d%m%Y%H%M")+'.csv', index=False)